In [6]:

import random
import numpy as np
def select_subset(array, subset_size):
    subset_indices = random.sample(range(len(array)), subset_size)
    subset = [array[i] for i in subset_indices]
    return np.array(subset)
origial_array=np.load('data/temp/sampled_coordinates.npy')
print(origial_array.shape)
coord_subset = select_subset(origial_array, 2000)
print(coord_subset.shape)


(5671, 3)
(2000, 3)


In [17]:
from functions.c_optimization_problem import initialize_dictionary
import numpy as np

Weight_Space = np.load("data/X_W_arrays/W_space.npy")

print(Weight_Space.shape)

(640, 438)


In [18]:
Dw = initialize_dictionary(Weight_Space, 20)
print(Dw.shape)

In [19]:


def initialize_coefficients(D, all_ellements, a):
    #X: (150, 500), d: (150X25), a: soothing factor
    coefficients=np.zeros((D.shape[1] , all_ellements.shape[1]))
    
    for j, sample in enumerate(all_ellements.T):
        for i, word in enumerate(D.T):
            diff = np.linalg.norm(sample-word)**2
            c_1=np.exp(-a * diff)
            coefficients[i][j]=c_1
    coefficients[:,j]/=np.sum(coefficients[:,j])
    return coefficients

(640, 20)


In [20]:
from functions.c_optimization_problem import initialize_coefficients

Lw=initialize_coefficients(Dw, Weight_Space, 0.2)
print(Lw.shape)


In [22]:
np.save("data/dictionaries/Dw", Weight_Space)
np.save("data/dictionaries/Lw", Lw)

In [31]:
def load_roi(subj_no):
    roi = np.load("data/MRI_MASKS/roi_masks_dataset/roi_"+ subj_no + ".npy")
    return roi 

roi = load_roi("9002430")
print(np.unique(roi), roi.shape)


def gather_roi_indices(roi_array):
    roi_inices=np.array(np.where(roi_array == 1)).T
    # print(roi_inices)
    roi_perimeter= [np.array(coord) for coord in roi_inices]
    return np.array(roi_perimeter)
# roi_perimeter=gather_roi_indices(roi)
# print(roi_perimeter.shape, roi_perimeter)


[0 1] (384, 384, 160)
(20481580, 3)


In [32]:
# np.save("data/temp/roi_perimeter", roi_perimeter)

In [34]:
roi_perimeter = np.load("data/temp/roi_perimeter.npy")
from sklearn.cluster import KMeans 


In [35]:
random_state=1
kmeans_model1=KMeans(n_clusters=120, verbose=False, init='k-means++', random_state=random_state)
kmeans_model1.fit(roi_perimeter)

KMeans(n_clusters=120, random_state=1, verbose=False)

In [36]:
kmeans_model2=KMeans(n_clusters=5, verbose=False, init='k-means++', random_state=random_state)
kmeans_model2.fit(kmeans_model1.cluster_centers_)

KMeans(n_clusters=5, random_state=1, verbose=False)

In [37]:
kmeans_model2.cluster_centers_

array([[215.70566424, 235.9783591 , 114.44476628],
       [219.66768373, 150.40789634,  79.20899779],
       [169.9209679 ,  85.58124149,  67.9128662 ],
       [210.95772957, 239.71583329,  40.72653948],
       [129.37661977, 253.17015378,  81.69693638]])

In [52]:
roi_subset = []
subregion_centers=kmeans_model2.cluster_centers_
for coordinate in roi_perimeter:
    x,y,z=coordinate
    if x%6==0 and y%6==0 and z%6==0:
        roi_subset.append(coordinate)
roi_subset=np.array(roi_subset)
print(roi_subset.shape)

(14548, 3)


In [56]:
first_label = kmeans_model1.predict(roi_subset)
final_label = kmeans_model2.predict(kmeans_model1.cluster_centers_[first_label])


In [57]:
subregion_1 = roi_subset[final_label==0]
subregion_2 = roi_subset[final_label==1]
subregion_3 = roi_subset[final_label==2]
subregion_4 = roi_subset[final_label==3]
subregion_5 = roi_subset[final_label==4]


In [58]:
print(subregion_1.shape, " -- ", subregion_2.shape, " -- ",subregion_3.shape, " -- ",subregion_4.shape, " -- ",subregion_5.shape)

(2725, 3)  --  (3381, 3)  --  (2987, 3)  --  (2658, 3)  --  (2797, 3)


In [62]:
np.save("data/temp/sub_1_coord.npy", subregion_1)
np.save("data/temp/sub_2_coord.npy", subregion_2)
np.save("data/temp/sub_3_coord.npy", subregion_3)
np.save("data/temp/sub_4_coord.npy", subregion_4)
np.save("data/temp/sub_5_coord.npy", subregion_5)

In [ ]:
coordinates= np.load("data/temp/general_coordinates.npy")
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
dir3 = [(os.listdir("data/MRI_MASKS/roi_masks_dataset")[i]).split(".")[0][4:] for i in range(len(os.listdir("data/MRI_MASKS/roi_masks_dataset")))]
common_subjects = list(set(dir1).intersection(dir2).intersection(dir3))

In [63]:
X = np.load("data/X_W_arrays/X_space.npy")
W = np.load("data/X_W_arrays/W_space.npy")

print(X.shape, W.shape)

(125, 437) (640, 438)


In [65]:
import os
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
common_subjects = list(set(dir1).intersection(dir2))
print(len(common_subjects))

437


In [66]:
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
common_subjects = list(set(dir1).intersection(dir2))
print(len(common_subjects))

437


In [71]:

# print(np.load("data/dictionaries/Dw.npy").shape)
Dx = initialize_dictionary(X, 25)
print(Dx.shape)
np.save("data/dictionaries/Dx.npy",Dx)

(125, 25)


In [74]:
Lx = initialize_coefficients(Dx, X, 0.8) #(25, 437)

In [76]:
np.save("data/dictionaries/Lx.npy", Lx)

In [77]:
import numpy as np
from functions.c_optimization_problem import *

W = np.load("data/X_W_arrays/W_space.npy")
Dw = initialize_dictionary(W, 25)
Lw = initialize_coefficients(Dw, W, 0.8)

In [78]:
np.save("data/dictionaries/W.npy", W)
np.save("data/dictionaries/Dw.npy", Dw)
np.save("data/dictionaries/LW.npy", Lw)

In [12]:
import numpy as np 
roi1=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9001104.npy')
roi2=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9023193.npy')
roi3=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9011420.npy')

coord=np.load('data/temp/sampled_coordinates.npy')

i=0

for coordinate in coord:
    x,y,z = coordinate
    if roi2[x,y,z] ==roi1 [x,y,z] == roi3 [x,y,z]:
        i+=1

print( i*100 / len(coord) , "%" )

35.07317933345089 %


# TESTING QCQP

In [4]:
def initialize_dictionary(all_elements, Dictionary_Size):
    
    k = Dictionary_Size
    descr_kmeans=KMeans(n_clusters=k, random_state=0)
    descr_kmeans.fit(all_elements.T)
    dictionary = descr_kmeans.cluster_centers_  

    return dictionary.T

#Soft voting to initialize Lx (Zhang et al 2013 - 2.Soft Assignment Coding)
def initialize_coefficients(D, all_ellements, a):
    #X: (150, 500), d: (150X25), a: soothing factor
    coefficients=np.zeros((D.shape[1] , all_ellements.shape[1]))
    
    for j, sample in enumerate(all_ellements.T):
        for i, word in enumerate(D.T):
            diff = np.linalg.norm(sample-word)**2 
            c_1=np.exp(-a * diff)
            coefficients[i][j]=c_1
    coefficients[:,j]/=np.sum(coefficients[:,j])
    return coefficients


In [8]:
import numpy as np
from scipy.optimize import minimize#, NonlinearConstraint 
from sklearn.cluster import KMeans 
# Objective function
def objective(dx, X, Lx):
    dx =dx.reshape(dx.shape[0],1)
    return np.linalg.norm(X - dx @ Lx, ord='fro')#**2

# Constraint function
def constraint(dx):
    return 1 - np.linalg.norm(dx, ord=2)

def constraint_jacobian(dx):
    return - (dx / np.linalg.norm(dx, ord=2))

X = np.random.rand(150, 500)
Dx_init = initialize_dictionary(X, 25)
Lx = initialize_coefficients(Dx_init, X, 0.8)



n = X.shape[1]
        
        
        # Constraint definition
constraint_definition = {'type': 'ineq', 'fun': constraint}     

def col_by_col_step(Dx_init):

    Dx = Dx_init.copy()
    
    for i, dx_init in enumerate(Dx_init.T):
        # Bounds for dx
        dx_init = np.reshape(dx_init, [dx_init.shape[0], 1])
        # constraint_definition = NonlinearConstraint(constraint, -np.inf, 1, jac=constraint_jacobian)
        # objective =  cp.Minimize(cp.sum_squares( + Di@L[i,:].reshape(1,no_samples)))
        A = X - np.sum([Dx[:,j].reshape(Dx[:,j].shape[0],1)@Lx[j,:].reshape(1,n) for j in range(Dx.shape[1]) if j!=i], axis=0)
        B = Lx[i, :].reshape(1,n)
        result = minimize(objective, dx_init, args=(A, B), method= 'trust-exact', constraints=[constraint_definition],jac = constraint_jacobian ,options={'maxiter' : 1000 , 'disp' : False })

        # Obtained solution
        dx_optimal = result.x
        
        # print(np.linalg.norm(dx_optimal))
        Dx[:,i] = dx_optimal

    return Dx

Dx = col_by_col_step(Dx_init)

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_11016\2090484645.py:39: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, dx_init, args=(A, B), method= 'trust-exact', constraints=[constraint_definition],jac = constraint_jacobian ,options={'maxiter' : 1000 , 'disp' : False })
c:\MySoftware\python3\Lib\site-packages\scipy\optimize\_minimize.py:565: RuntimeWarning: Method trust-exact cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


ValueError: Hessian matrix is required for trust region exact minimization.

In [5]:
def optimization_loop(Dx_init):
    norms =[]
    Dx = Dx_init.copy()
    for i in range(100): 
        Dx = col_by_col_step(Dx)
        print(np.linalg.norm(X - Dx @ Lx, ord='fro'))
    return Dx

print('before' , np.linalg.norm(X - Dx @ Lx, ord='fro'))


Dx = optimization_loop(Dx_init)


print('after' , np.linalg.norm(X - Dx @ Lx, ord='fro'))
        

before 157.85246210757225


C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_11016\1358110505.py:39: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, dx_init, args=(A, B), method='SLSQP', constraints=[constraint_definition], options={'maxiter' : 1000 , 'disp' : False })


157.85246210757225
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996
157.85097152284996


KeyboardInterrupt: 

# LASSO

In [6]:
import numpy as np
from sklearn.cluster import KMeans 

W = np.random.uniform(0,1, [645, 500])
Dw = initialize_dictionary(W, 25)
Lx = np.random.uniform(0,1, [25, 500])
M = np.random.uniform(0,1, [25, 25])

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [7]:
import numpy as np 
from sklearn.linear_model import LassoLars
 

def lars_col_by_col(X, y , alpha=0.01):
    lasso = LassoLars(alpha=alpha)
    lasso.fit(X, y)
    return lasso.coef_


def update_Lw_step(W, Dw, M, Lx, gamma=0.7, alpha=0.01):

    Y = np.vstack([W, gamma * M @ Lx])
    X = np.vstack([Dw, gamma * np.identity(25)])
    Lw = np.zeros([Dw.shape[1], W.shape[1]])
    
    for i, y in enumerate(Y.T):
    
        lw_col = lars_col_by_col(X, y)
        Lw[:, i] = lw_col

    return Lw

Lw = update_Lw_step(W, Dw, M, Lx, gamma=0.7, alpha=0.01)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()